In [1]:
%load_ext autoreload
%autoreload 2

## 이전 작업에서 생성한 S3 Bucket의 이름을 `bucket` 변수에 입력하시기 바랍니다.

- `bucket`

In [15]:
bucket='YOUR_BUCKET_NAME'

## 트레이닝에 사용할 Docker 이미지의 주소를 확인합니다.

>그외 다른 알고리즘 및 이미지들의 주소는 [이곳](https://docs.aws.amazon.com/ko_kr/sagemaker/latest/dg/sagemaker-algo-docker-registry-paths.html)에서 확인하실수 있습니다.

In [3]:
import boto3
from sagemaker import get_execution_role

role = get_execution_role()

containers = {'us-west-2': '433757028032.dkr.ecr.us-west-2.amazonaws.com/image-classification:latest',
              'us-east-1': '811284229777.dkr.ecr.us-east-1.amazonaws.com/image-classification:latest',
              'us-east-2': '825641698319.dkr.ecr.us-east-2.amazonaws.com/image-classification:latest',
              'eu-west-1': '685385470294.dkr.ecr.eu-west-1.amazonaws.com/image-classification:latest'}
training_image = containers[boto3.Session().region_name]
print(training_image)

811284229777.dkr.ecr.us-east-1.amazonaws.com/image-classification:latest


## Training 파라미터들

### 1. Training job 파라미터

- Input specification: `"InputDataConfig"` training 및 validation의 데이터의 위치, 데이터 타입은
`"ContentType"`으로 설정할 수 있으며 설정 할 수 있는 값은 `"application/x-recordio"` 또는 `"application/x-image"`입니다.
- Output specification: `"OutputDataConfig"` 섹션에 정의 되어 있으며 결과물을 저장할 곳을 정의 합니다.
- Resource config: 인스턴스 타입 및 대수, `"InstanceCount"`가 1보다 크면 분산 처리합니다.

### 2. 알고리즘 hyperparameters

- `num_layers` : 신경 네트워크의 layers 개수 혹은 네트워크의 depth
- `num_training_samples` : Training 샘플의 총 개수, CIFAR-10 dataset의 경우 50000
- `num_classes` : 총 분류 카테고리의 수, 사용하는 알고리즘의 경우 Imagenet의 1000개의 카테고리로 트레이닝 되었습니다. Fine-tuning의 데모로 이번 가이드에서는 CIFAR-10의 10개의 카테고리를 사용합니다.
- `epochs` : Training epochs 수
- `learning_rate` : Training learning rate
- `mini_batch_size` : mini batch에 사용하는 training 샘플의 개수, 분산 training의 경우 training 샘플의 숫자는 N * mini_batch_size 입니다. 이때, N은 training을 하는 호스트의 대수 입니다.

위의 파라미터들을 설정 후 training job을 실행합니다. training이 끝나는 status를 확인합니다.
이 실습의 경우 `p2.xlarge` 1대를 사용 하였을 때 대략 20분 정도 소요 됩니다. 


## 아마존이 제공하는 알고리즘은 Layer의 수를 18, 34, 50, 101, 152, 200을 지원합니다.

>이번 트레이닝에서는 18 Layer를 사용합니다.

In [14]:
# The algorithm supports multiple network depth (number of layers). They are 18, 34, 50, 101, 152 and 200
# For this training, we will use 18 layers
num_layers = 18

## 입력 이미지의 사이즈를 설정 합니다.

> CIFAR의 경우 28x28x3 입니다.

In [5]:
# we need to specify the input image shape for the training data
image_shape = "3,28,28"

## Training와 관련한 파라 미터들을 설정합니다.

1. Training 샘플 수
2. Class 수
3. Mini Batch 사이즈
4. Epoch 수
5. Learning Rate

In [6]:
# we also need to specify the number of training samples in the training set
# for CIFAR-10 it is 50000
num_training_samples = 50000
# specify the number of output classes
num_classes = 10
# batch size for training
mini_batch_size =  128
# number of epochs
epochs = 10
# learning rate
learning_rate = 0.01

## Pre-trainined Weights를 사용 할 지 여부에 대한 Flag를 설정합니다.

>시간 및 효율성을 위해서 1로 설정합니다.

In [7]:
# Since we are using transfer learning, we set use_pretrained_model to 1 so that weights can be 
# initialized with pre-trained weights
use_pretrained_model = 1

## 인스턴스 타입을 설정합니다.

> Amazon built-in `Image Classification` 알고리즘의 경우 "ml.c4.xlarge" 사용시 에러를 발생합니다.

> 그외 Sagemaker에서 "ml.c5.xlarge" 사용 하시려고 할 경우 Limit 해제 요청을 하셔야 합니다.

In [8]:
instance_type="ml.p2.xlarge"

## Unique job name을 설정합니다.

>`job_name` 변수는 이후 계속 사용합니다.

In [9]:
import boto3
from time import gmtime, strftime
s3 = boto3.client('s3')

# create unique job name 
job_name_prefix = 'sagemaker-imageclassification-cifar10'
timestamp = strftime('-%Y-%m-%d-%H-%M-%S', gmtime())
job_name = job_name_prefix + timestamp

print("Your job name is\n\njob_name=\"{}\"\n".format(job_name))

Your job name is

job_name="sagemaker-imageclassification-cifar10-2018-01-19-13-58-21"



## Api 호출을 위한 parmeter template을 생서합니다.

> Tensorflow 혹은 mxnet을 사용하실 경우 Training API 호출 방식이 차이가 있으므로 해당 `tensorflow` 혹은 `mxnet` 예제를 확인 하시기 바랍니다. 

In [10]:
training_params = \
{
    # specify the training docker image
    "AlgorithmSpecification": {
        "TrainingImage": training_image,
        "TrainingInputMode": "File"
    },
    "RoleArn": role,
    "OutputDataConfig": {
        "S3OutputPath": 's3://{}/{}/output'.format(bucket, job_name_prefix)
    },
    "ResourceConfig": {
        "InstanceCount": 1,
        "InstanceType": "{}".format(instance_type),
        "VolumeSizeInGB": 50
    },
    "TrainingJobName": job_name,
    "HyperParameters": {
        "image_shape": image_shape,
        "num_layers": str(num_layers),
        "num_training_samples": str(num_training_samples),
        "num_classes": str(num_classes),
        "mini_batch_size": str(mini_batch_size),
        "epochs": str(epochs),
        "learning_rate": str(learning_rate),
        "use_pretrained_model": str(use_pretrained_model)
    },
    "StoppingCondition": {
        "MaxRuntimeInSeconds": 360000
    },
#Training data should be inside a subdirectory called "train"
#Validation data should be inside a subdirectory called "validation"
#The algorithm currently only supports fullyreplicated model (where data is copied onto each machine)
    "InputDataConfig": [
        {
            "ChannelName": "train",
            "DataSource": {
                "S3DataSource": {
                    "S3DataType": "S3Prefix",
                    "S3Uri": 's3://{}/train/cifar10_train.rec'.format(bucket),
                    "S3DataDistributionType": "FullyReplicated"
                }
            },
            "ContentType": "application/x-recordio",
            "CompressionType": "None"
        },
        {
            "ChannelName": "validation",
            "DataSource": {
                "S3DataSource": {
                    "S3DataType": "S3Prefix",
                    "S3Uri": 's3://{}/validation/cifar10_val.rec'.format(bucket),
                    "S3DataDistributionType": "FullyReplicated"
                }
            },
            "ContentType": "application/x-recordio",
            "CompressionType": "None"
        }
    ]
}
print('Training job name: {}'.format(job_name))
print('\nInput Data Location: {}'.format(training_params['InputDataConfig'][0]['DataSource']['S3DataSource']))

Training job name: sagemaker-imageclassification-cifar10-2018-01-19-13-58-21

Input Data Location: {'S3DataType': 'S3Prefix', 'S3Uri': 's3://awskrug-sagemaker-demo/train/cifar10_train.rec', 'S3DataDistributionType': 'FullyReplicated'}


## 위에서 생성한 파라미터를 이용하여 Sagemaker Api 호출을 합니다.

In [11]:
# create the Amazon SageMaker training job
sagemaker = boto3.client(service_name='sagemaker')
sagemaker.create_training_job(**training_params)

{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
   'content-length': '132',
   'content-type': 'application/x-amz-json-1.1',
   'date': 'Fri, 19 Jan 2018 13:58:21 GMT',
   'x-amzn-requestid': 'ec5d0bb6-6e08-4f2e-83fd-cb434e077771'},
  'HTTPStatusCode': 200,
  'RequestId': 'ec5d0bb6-6e08-4f2e-83fd-cb434e077771',
  'RetryAttempts': 0},
 'TrainingJobArn': 'arn:aws:sagemaker:us-east-1:936074911029:training-job/sagemaker-imageclassification-cifar10-2018-01-19-13-58-21'}

## Training Job에 대한 상태 확인

In [12]:
# confirm that the training job has started
status = sagemaker.describe_training_job(TrainingJobName=job_name)['TrainingJobStatus']
print('Training job current status: {}'.format(status))

Training job current status: InProgress


## Training Job 종료 확인

In [16]:
try:
    # wait for the job to finish and report the ending status
    sagemaker.get_waiter('training_job_completed_or_stopped').wait(TrainingJobName=job_name)
    training_info = sagemaker.describe_training_job(TrainingJobName=job_name)
    status = training_info['TrainingJobStatus']
    print("Training job ended with status: " + status)
    print("\nUse job_name variable next notebook\n\njob_name=\"{}\"\n".format(job_name))
except:
    print('Training failed to start')
     # if exception is raised, that means it has failed
    message = sagemaker.describe_training_job(TrainingJobName=job_name)['FailureReason']
    print('Training failed with the following error: {}'.format(message))

Training job ended with status: Completed

Use job_name variable next notebook

job_name="sagemaker-imageclassification-cifar10-2018-01-19-13-58-21"



>  아래 출력의 변수의 `job_name` 선언 부분을 복사해 놓으시기 바랍니다.